# Table of Contents
 <p>

Script to connect to cloud storage upload a tif and then transfer it from storage to gee

In [110]:
import ee
from google.cloud import storage 
import os
import rasterio

class assetManagement(object):
    ## Connects to bucket, upload the image and once it is ready transfers it to GEE associated collection.
    def __init__(self,imageObject):
        """checks the image and sets up the properties """
        self.meta=imageObject
        self.imageNames=self.getImageName()
        self.sources = []
        
    def getImageName(self):
        "gets the listof names from sources"
        return [os.path.basename(name) for name in self.meta['sources']]
    
    def checksImages(self):
        "Checks the images that we will compose have the same nº bands as they are going to become one image and part of the image collection"
        metadata=[]
        for image in self.meta['sources']:
            with rasterio.open(image) as src:
                metaData=src.meta
                
                assert metaData['driver'] == 'GTiff', "Driver is not supported: {0}".format(metaData['driver'])
                assert metaData['count'] == len(self.meta['bandNames']), "Nbands incorrect, expected: {0}, {1} provided".format(metaData['count'],len(self.meta['bandNames']))
                
                metadata.append({'dtype': metaData['dtype'], 'driver': metaData['driver'], 'nodata': metaData['nodata'], 'nBands': metaData['count'],'crs': src.crs.to_string()})
        
        assert len(set([item['dtype'] for item in metadata])) == 1, "Images list dtypes aren't compatibles. Expected: 1, {1} provided".format(metaData['count'],len(set([item['dtype'] for item in metadata])))
        assert len(set([item['driver'] for item in metadata])) == 1, "Images list drivers aren't compatibles. Expected: 1, 1 provided".format(metaData['count'],len(set([item['driver'] for item in metadata])))
        assert len(set([item['nodata'] for item in metadata])) == 1, "Images list nodata values aren't compatibles. Expected: 1, {1} provided".format(metaData['count'],len(set([item['nodata'] for item in metadata])))
        assert len(set([item['nBands'] for item in metadata])) == 1, "Images list nBands number aren't compatibles. Expected: 1, {1} provided".format(metaData['count'],len(set([item['nBands'] for item in metadata])))
        assert len(set([item['crs'] for item in metadata])) == 1, "Images list crs aren't compatibles. Expected: 1, {1} provided".format(metaData['count'],len(set([item['crs'] for item in metadata])))       
        return metadata[0]
                    
    
    def setUpCredentials(self):
        """Sets up the credentials"""
        credentials = ee.ServiceAccountCredentials(os.getenv(service_account), 'privatekey.json')
        ee.Initialize(credentials)
        #ee.Initialize()
        storage_client=storage.Client.from_service_account_json('Resource Watch-da6fcba3f7f6.json')
        return storage_client.get_bucket(self.meta['gcsBucket'])
    
    def uploadGCS(self, imageName):
        "Upload the image to google cloud storage"
        imageIndex = self.imageNames.index(imageName)
        blob = self.gcsBucket.blob('{0}/{1}'.format(self.meta['collectionAsset'],imageName))
        blob.upload_from_filename(self.meta['sources'][imageIndex])
        blob.make_public()
        
        return {'primaryPath': 'gs://{gcsBucket}/{collectionName}/{imageNa}'.format(gcsBucket=self.meta['gcsBucket'],collectionName=self.meta['collectionAsset'],imageNa=imageName)}
        
    def transferGEE(self):
        "Transfers the images from google cloud storage to gee asset"
        task_id = ee.data.newTaskId()[0]
        request = {
            'id':'{collectionAsset}/{assetName}'.format(collectionAsset= self.meta['collectionAsset'],assetName =self.meta['assetName']),
            'properties':self.meta['properties'],
            'tilesets': [{'sources': self.sources}],
            'pyramidingPolicy':self.meta['pyramidingPolicy'].upper(),
            'bands':self.meta['bandNames']
        }
        print(request)
        ee.data.startIngestion(task_id, request, True)
        return task_id
    
    def execute(self):
        #Checks if the images are correct
        self.checksImages()
        
        #sets up credentials
        self.gcsBucket=self.setUpCredentials()
        
        #Uploads file/s to GCS
        self.sources = list(map(self.uploadGCS, self.imageNames))
        
        #Transfers it from GCS to GEE
        task_id = self.transferGEE()
        
        print('TaskID: {0}'.format(task_id))
        print('Status: {0}'.format(ee.data.getTaskStatus(task_id)[0]))
        return 0
        
        

In [111]:
myImage={
    'sources':['/Users/alicia/Downloads/results%2Fhistorical%2Fdecadal_test_historical_1991_2000_hdds.tiff'],
    'gcsBucket':'gee-image-transfer',
    'collectionAsset':'users/Aliciaarenzana/testcollection',
    'assetName':'t2000',
    'bandNames':[{'id': 'R'}, {'id': 'G'}, {'id': 'B'}],
    'pyramidingPolicy':'MODE',
    'properties':{
        'my_imageProperties':'to add to the collection'
    }
    
}

In [112]:
test= assetManagement(myImage)
print(test.checksImages())
test.execute()

{'dtype': 'float32', 'driver': 'GTiff', 'nodata': None, 'nBands': 3, 'crs': '+init=epsg:4326'}
{'id': 'users/Aliciaarenzana/testcollection/t2000', 'properties': {'my_imageProperties': 'to add to the collection'}, 'tilesets': [{'sources': [{'primaryPath': 'gs://gee-image-transfer/users/Aliciaarenzana/testcollection/results%2Fhistorical%2Fdecadal_test_historical_1991_2000_hdds.tiff'}]}], 'pyramidingPolicy': 'MODE', 'bands': [{'id': 'R'}, {'id': 'G'}, {'id': 'B'}]}
TaskID: DA5WIU4LFBT7L7AACGME5SBU
Status: {'id': 'DA5WIU4LFBT7L7AACGME5SBU', 'state': 'READY', 'creation_timestamp_ms': 1514388886298, 'update_timestamp_ms': 1514388886298, 'description': 'Asset ingestion: users/Aliciaarenzana/testcollection/t2000', 'task_type': 'INGEST'}


0

### Upload file to google cloud storage

In [9]:
# Imports the Google Cloud client library

# Instantiates a client
storage_client = storage.Client.from_service_account_json('Resource Watch-da6fcba3f7f6.json')
# The name for the new bucket
bucket_name = 'gee-image-transfer'
bucket = storage_client.get_bucket(bucket_name)
# The upload!
filename='test-collection/results_rcp85_30_y_test_rcp85_2021_2050_hdds.tiff'
filepath='/Users/alicia/Downloads/results_rcp85_30_y_test_rcp85_2021_2050_hdds.tiff'
blob = bucket.blob(filename)

blob.upload_from_filename(filepath)
blob.make_public()




### Transfer file from GCS to GEE

In [7]:
#service_account = 'my-service-account@...gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
#ee.Initialize(credentials)

ee.Initialize()
# Create task
task_id = ee.data.newTaskId()[0]
# Transfer from gcs to gee
"""
params: The object that describes the import task, which can
        have these fields:
          id (string) The destination asset id (e.g. users/foo/bar).
          tilesets (array) A list of Google Cloud Storage source file paths
            formatted like:
              [{'sources': [
                  {'primaryPath': 'foo.tif', 'additionalPaths': ['foo.prj']},
                  {'primaryPath': 'bar.tif', 'additionalPaths': ['bar.prj'},
              ]}]
            Where path values correspond to source files' Google Cloud Storage
            object names, e.g. 'gs://bucketname/filename.tif'
          bands (array) An optional list of band names formatted like:
            [{'id': 'R'}, {'id': 'G'}, {'id': 'B'}]
            
    ie: 'id':'users/Aliciaarenzana/testcollection/results_rcp85_30_y_test_rcp85_2021_2050_hdds',
    'properties':{'test':'test2'},
    'tilesets': [{'sources': [{'primaryPath': 'gs://gee-image-transfer/test-collection/results_rcp85_30_y_test_rcp85_2021_2050_hdds.tiff'}],
                 'fileBands':[{'fileBandIndex': -1, 'maskForAllBands': True}]}],
    'pyramidingPolicy':'MODE',
    'missingData':5,
    'crs':'crs',
    'bands':[{'id': 't25','pyramidingPolicy':'','missingData':5}, {'id': 't50','pyramidingPolicy':'','missingData':5}, {'id': 't75','pyramidingPolicy':'','missingData':5}]

"""
request = {
    'id':'users/Aliciaarenzana/testcollection/results_rcp85_30_y_test_rcp85_2021_2050_hdds',
    'properties':{'test':'test2'},
    'tilesets': [{'sources': [{'primaryPath': 'gs://gee-image-transfer/test-collection/results_rcp85_30_y_test_rcp85_2021_2050_hdds.tiff'}]}],
    'pyramidingPolicy':'MODE',
    'bands':[{'id': 't25'}, {'id': 't50'}, {'id': 't75'}]
}

ee.data.startIngestion(task_id, request, True)

# setup acl properties


In [82]:
acl= {
          "all_users_can_read" : True
      }
asset_id='users/Aliciaarenzana/testcollection'
ee.data.setAssetAcl(asset_id, json.dumps(acl))

print(ee.data.getAssetAcl(asset_id))

{'all_users_can_read': True, 'readers': [], 'writers': [], 'owners': ['alicia.arenzana@gmail.com']}


In [6]:
# List buckets

buckets = list(storage_client.list_buckets())
print(buckets)

[<Bucket: api-backups>, <Bucket: gee-image-transfer>, <Bucket: gee-tiles>, <Bucket: nex-gddp>, <Bucket: nexgddp-tiles>, <Bucket: rw-usage-export>]
